In [1]:
bucket = spark._jsc.hadoopConfiguration().get("fs.gs.system.bucket")
url = "gs://" + bucket + "/data/chicago-taxi-trip/cleaned-chicago-taxi/"

In [2]:
from pyspark.sql.types import (StructType, 
                               StructField, 
                               DoubleType,
                               IntegerType,
                               StringType,
                               TimestampType)

taxi_schema = StructType([StructField('unique_key', StringType(), True),
                StructField('taxi_id', StringType(), True),
                StructField('trip_start_timestamp', TimestampType(), True), 
                StructField('trip_end_timestamp', TimestampType(), True), 
                StructField('trip_seconds', IntegerType(), True), 
                StructField('trip_miles', IntegerType(), True), 
                StructField('pickup_census_tract', StringType(), True), 
                StructField('dropoff_census_tract', StringType(), True), 
                StructField('pickup_community_area', IntegerType(), True), 
                StructField('dropoff_community_area', IntegerType(), True), 
                StructField('fare', IntegerType(), True), 
                StructField('tips', IntegerType(), True), 
                StructField('tolls', IntegerType(), True), 
                StructField('extras', IntegerType(), True), 
                StructField('trip_total', IntegerType(), True), 
                StructField('payment_type', StringType(), True), 
                StructField('company', StringType(), True), 
                StructField('pickup_latitude', StringType(), True), 
                StructField('pickup_longitude', StringType(), True), 
                StructField('pickup_location', StringType(), True), 
                StructField('dropoff_latitude', StringType(), True), 
                StructField('dropoff_longitude', StringType(), True), 
                StructField('dropoff_location', StringType(), True)])

taxi = spark.read.format("csv").option("header", "true").schema(taxi_schema).csv(url)

In [4]:
taxi.show(5)

+--------------------+--------------------+--------------------+-------------------+------------+----------+-------------------+--------------------+---------------------+----------------------+----+----+-----+------+----------+-------------+--------------------+---------------+----------------+--------------------+----------------+-----------------+----------------+
|          unique_key|             taxi_id|trip_start_timestamp| trip_end_timestamp|trip_seconds|trip_miles|pickup_census_tract|dropoff_census_tract|pickup_community_area|dropoff_community_area|fare|tips|tolls|extras|trip_total| payment_type|             company|pickup_latitude|pickup_longitude|     pickup_location|dropoff_latitude|dropoff_longitude|dropoff_location|
+--------------------+--------------------+--------------------+-------------------+------------+----------+-------------------+--------------------+---------------------+----------------------+----+----+-----+------+----------+-------------+------------------